# Deploy a model from Azure Databricks to [Managed Online Endpoints](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-online-endpoints?tabs=python). 
In this notebook we will see how to deploy a trained model on Azure Databricks to Managed Online Endpoints securely created on Azure Machine Learning.

With this approach, we can use the best of Databricks and Azure Machine Learning, training robust models and taking full advantage of this integration's scalability and security.

In this deployment model, we will use the [Python SDK v2](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-ml-readme?view=azure-python) to integrate with Azure ML in a simple and efficient way. To ensure security, we store security keys and sensitive information such as ``subscription_id``
, ``resource_group``, and ``AML workspace name`` in an [Azure Key Vault](https://learn.microsoft.com/en-us/azure/key-vault/general/basic-concepts).

In [0]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

subscription_id = dbutils.secrets.get(scope = 'akd-adb-prv', key = 'SUBSCRIPTION-ID')
resource_group = dbutils.secrets.get(scope = 'akd-adb-prv', key = 'RESOURCE-GROUP')
workspace = dbutils.secrets.get(scope = 'akd-adb-prv', key = 'AML-WORKSPACE-NAME')

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

Now we need to define the Managed Online Endpoint endpoint's configs. 

We set the ``publick_network_access`` as ``disabled`` to ensure that the inbound (scoring) access will use the private endpoint of the [Azure ML Workspace](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-configure-private-link).

In [0]:
from azure.ai.ml.entities import (ManagedOnlineEndpoint, 
                                 ManagedOnlineDeployment, 
                                 Model, 
                                 CodeConfiguration, 
                                 Environment)

endpoint_name = 'churn-endpoint-01' # YOUR ENDPOINT NAME

endpoint = ManagedOnlineEndpoint(name=endpoint_name,  
                         description='Realtime endpoint to predict churn', 
                         tags={'model': 'XGBoost'}, 
                         auth_mode="key", 
                         public_network_access="disabled" 
)

ml_client.online_endpoints.begin_create_or_update(endpoint)

We need to wait for the ``provisioning_state`` to be **Succeeded**. You can define the ``sleep_seconds`` and ``max_loop`` variables to wait for more cycles/seconds according to the time you need to wait.

In [0]:
import time

max_loop = 50
num_loop = 0
sleep_seconds = 10

provisioning_state = ml_client.online_endpoints.get(endpoint_name).provisioning_state

while provisioning_state != 'Succeeded' and num_loop < max_loop:
    print(f'Checking the endpoint creation. Interaction {num_loop}')
    time.sleep(sleep_seconds)
    provisioning_state = ml_client.online_endpoints.get(endpoint_name).provisioning_state
    num_loop+=1

print(f'The endpoint {endpoint_name} was created with {provisioning_state} provision_state')

Next, we will copy the Databricks model to a temp folder on DBFS. In our case, we persisted the model to Azure Data Lake Storage before, so we are copying from the ADLS to DBFS. You can also copy directly from mlflow to DBFS. However, it's important to have this staging folder to access from AML SDK.

We need to copy the ``conda.yml`` file and have a ``score.py`` (entry script) in the staging folder as well. These files are used to define the dependencies (libraries) that the endpoint needs to consider (conda.yml) and also the entry script that will receive the data submitted to the endpoint and will pass it to the model (score.py). Here you can customize it according to your process as well. For more information about that, please look at this doc about [advanced entry script authoring](https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-deploy-advanced-entry-script).

In [0]:
spark.conf.set(
    f"fs.azure.account.key.{dbutils.secrets.get(scope = 'akd-adb-prv', key = 'STORAGE-STAGING-NAME')}.dfs.core.windows.net",
    dbutils.secrets.get(scope = 'akd-adb-prv', key = 'STORAGE-STAGING-KEY'))

dbutils.fs.cp(f'abfs://model-data@{dbutils.secrets.get(scope = 'akd-adb-prv', key = 'STORAGE-STAGING-NAME')}.dfs.core.windows.net/AML-Debug-ME/churn-deploy/environment/conda.yml', 'dbfs:/tmp/model/conda.yml')
dbutils.fs.cp(f'abfs://model-data@{dbutils.secrets.get(scope = 'akd-adb-prv', key = 'STORAGE-STAGING-NAME')}.dfs.core.windows.net/AML-Debug-ME/churn-deploy/onlinescoring/score.py', 'dbfs:/tmp/model/onlinescoring/score.py')
dbutils.fs.cp(f'abfs://model-data@{dbutils.secrets.get(scope = 'akd-adb-prv', key = 'STORAGE-STAGING-NAME')}.dfs.core.windows.net/AML-Debug-ME/churn-deploy/model', 'dbfs:/tmp/model/model', True)

We create a Model on Azure Machine Learning with all models' artifacts

In [0]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

cloud_model = Model(
    path='/dbfs/tmp/model/model',
    name="churn-model-adb",
    type=AssetTypes.CUSTOM_MODEL,
    description="Model created from cloud path.",
)

model = ml_client.models.create_or_update(cloud_model)

And the last stage is to deploy the model according to the required configs.

Here we need to define the ``instance_type`` and how many nodes we would like to use (``instance_count``).

We set the ``egress_public_network_access`` to ``disabled`` as well to ensure that the communication between a deployment and external resources, including Azure resources, will use the private endpoint.

In [0]:
deployment_name = 'blue'

env = Environment(
    conda_file="/dbfs/tmp/model/conda.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

deployment = ManagedOnlineDeployment(name=deployment_name, 
                                          endpoint_name=endpoint_name, 
                                          model=model, 
                                          code_configuration=CodeConfiguration(code='/dbfs/tmp/onlinescoring/',
                                                                               scoring_script='score.py'),
                                          environment=env, 
                                          instance_type='Standard_DS2_v2', 
                                          instance_count=1, 
                                          egress_public_network_access="disabled"
)

ml_client.online_deployments.begin_create_or_update(deployment=deployment)

Wait for the deployment succeed

In [0]:
max_loop = 100
sleep_seconds = 30
num_loop = 0

provisioning_state = ml_client.online_deployments.get(name = deployment_name, endpoint_name=endpoint_name).provisioning_state

while provisioning_state != 'Succeeded' and num_loop < max_loop:
    print(f'Checking the deployment. Interaction {num_loop}')
    time.sleep(sleep_seconds)
    provisioning_state = ml_client.online_deployments.get(name = deployment_name, endpoint_name=endpoint_name).provisioning_state
    num_loop+=1

print(f'The deployment for endpoint {endpoint_name} has finished with {provisioning_state} provision_state')

Finally, with the endpoint, we can make an API request to ensure we can get the result. Notice that this request will be done in a private context as well.

In [0]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
data = {"data": [{"Idade": 21,
                                   "RendaMensal": 9703, 
                                   "PercentualUtilizacaoLimite": 1.0, 
                                   "QtdTransacoesNegadas": 5.0, 
                                   "AnosDeRelacionamentoBanco": 12.0, 
                                   "JaUsouChequeEspecial": 0.0, 
                                   "QtdEmprestimos": 1.0, 
                                   "NumeroAtendimentos": 100, 
                                   "TMA": 300, 
                                   "IndiceSatisfacao": 2, 
                                   "Saldo": 6438, 
                                   "CLTV": 71}
]}

body = str.encode(json.dumps(data))

url = 'https://churn-endpoint-01.southcentralus.inference.ml.azure.com/score'

api_key = dbutils.secrets.get(scope = 'akd-adb-prv', key = 'AML-ENDPOINT-KEY')

if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'blue' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))